# Démonstration #


Dans cette démonstration notre modèle va prédire les annulations de réservation d'hôtels avec un classificateur binaire.

C'est à dire qu'il va déterminer si oui ou non une réservation a été annulé.

In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

hotel = pd.read_csv('../input/dl-course-data/hotel.csv')

# En X on a notre jeu de données avec nos features et notre cible est la colonne 'is canceled'
X = hotel.copy()
y = X.pop('is_canceled')

X['arrival_date_month'] = \
    X['arrival_date_month'].map(
        {'January':1, 'February': 2, 'March':3,
         'April':4, 'May':5, 'June':6, 'July':7,
         'August':8, 'September':9, 'October':10,
         'November':11, 'December':12}
    )

features_num = [
    "lead_time", "arrival_date_week_number",
    "arrival_date_day_of_month", "stays_in_weekend_nights",
    "stays_in_week_nights", "adults", "children", "babies",
    "is_repeated_guest", "previous_cancellations",
    "previous_bookings_not_canceled", "required_car_parking_spaces",
    "total_of_special_requests", "adr",
]
features_cat = [
    "hotel", "arrival_date_month", "meal",
    "market_segment", "distribution_channel",
    "reserved_room_type", "deposit_type", "customer_type",
]

transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"), # there are a few missing values
    StandardScaler(),
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown='ignore'),
)

preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)

X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, stratify=y, train_size=0.75)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

input_shape = [X_train.shape[1]]

Nous allons créer un réseau de neurone tel que le schéma ci-dessous

<figure style="padding: 1em;">
<img src="https://i.imgur.com/V04o59Z.png" width="400" alt="Diagram of network architecture: BatchNorm, Dense, BatchNorm, Dropout, Dense, BatchNorm, Dropout, Dense.">
<figcaption style="textalign: center; font-style: italic"><center>Diagram of a binary classifier.</center></figcaption>
</figure>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(256, activation='relu', input_shape=[33]),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'), 
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid'),
])

- La fonction Dropout supprime un pourcentage de neurone de la couche suivante 
- La fonction BatchNormalization permet d'aider si l'apprentissage est lent ou instable (exemple : avant d'utiliser des données ont les nettoies, de cette même manière on normalise notre réseaux de neurone avant de passer les données dedans, on peut dire que le BatchNormalization est un peu une bonne pratique)


In [ ]:
model.summary()

- Total params : ce sont les poids du notre réseaux de neurone

- Trainable params : ce sont l'ensemble des paramètre qui peuvent être modifier par Tensorflow automatiquement lors de l'apprentissage dans le but de minimiser l'erreur que l'on va définir

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

- optimizer : c'est le type d'optimizer qui permet de réduire l'erreur, l'adam est une variante de la descente de gradient stochastique mais qui adapte le learning rate en temps réel 
- loss : fonction d'erreur, qui est içi la crossentropy binaire
- metric : c'est la précision de notre entrainement

In [ ]:
# En pratique plus on fais d'itération (d'epoch) et plus l'erreur est censé diminuer, 
# La fonction early stop permet d'arrêter l'apprentissage au moment ou l'erreur commence de nouveau à augmenter cela évite donc de continuer a utiliser de la puissance de calcul mais surtout d'éviter le surapprentissage
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],
)

On voit que grâce à la fonction EarlyStop, l'apprentissage s'est stopper à 27 epoch

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

- En orange on a la courbe qui correspond au jeu de test 
- En bleu la courbe qui correspond au jeu d'apprentissage